In [1]:
%%time

import datetime as dt
import os

import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

from common.data_frame_functions.functions import Functions

functions: Functions = Functions()

CPU times: total: 2.66 s
Wall time: 22.5 s


In [2]:
%%time

# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture",	"trippinthroughtime",
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"fatsquirrelhate",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks",
	"AesPleasingAsianGirls"
]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion","AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 4.54 ms


In [4]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]


CPU times: total: 391 ms
Wall time: 5.37 s


In [5]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

'==== CURATED DATA ===='

(37031, 14)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,True,True,[]
13d661b,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,True,True,[]
13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,True,False,[]


CPU times: total: 516 ms
Wall time: 5.42 s


In [6]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

'==== UPDATED EXTANT DATA ===='

(41505, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,131jvcx.jpg,data/image/131jvcx.jpg,PrettyGirlDiffusion,True,True,False,[]


CPU times: total: 359 ms
Wall time: 6.48 s


In [7]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

CPU times: total: 125 ms
Wall time: 468 ms


In [8]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 18/18 [00:00<00:00, 58.95it/s]


In [9]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				for submission in tqdm(subreddit_stream, total=len(subreddit_stream),
									   desc=f"Posts - {sub} - {time_filter}"):
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

day: 100%|██████████| 18/18 [00:15<00:00,  1.16it/s]


In [10]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

Reading Temp Dir For Subs...: 100%|██████████| 18/18 [00:03<00:00,  5.03it/s]


'==== TEMP DATA ===='

(2532, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13a5o5w,13a5o5w,SFWRedheads,Sofia_Red_Fox,Am I dateable? 💕,,,/r/SFWRedheads/comments/13a5o5w/am_i_dateable/,https://i.imgur.com/V4aYZoC.jpg,13a5o5w.jpg,,,False,False,False,[]
13a63qc,13a63qc,SFWRedheads,Bakedredhead,Sunsets look best on redheads,,,/r/SFWRedheads/comments/13a63qc/sunsets_look_b...,https://i.redd.it/hnmnfup79cya1.jpg,13a63qc.jpg,,,False,False,False,[]
13aa5bw,13aa5bw,SFWRedheads,Redhotwife88,Just a Mom in Regular Clothes! [F],,,/r/SFWRedheads/comments/13aa5bw/just_a_mom_in_...,https://i.imgur.com/hUXcket.jpg,13aa5bw.jpg,,,False,False,False,[]
13ae244,13ae244,SFWRedheads,evelynclaire_xoxo,Fully body shot... thoughts?,,,/r/SFWRedheads/comments/13ae244/fully_body_sho...,https://i.redd.it/irgp2rclicya1.jpg,13ae244.jpg,,,False,False,False,[]
13af0tb,13af0tb,SFWRedheads,itsSavannahWilde,About to hit the gym. Got any good weekend plans?,,,/r/SFWRedheads/comments/13af0tb/about_to_hit_t...,https://i.redd.it/behzhc90rcya1.jpg,13af0tb.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13huk92,13huk92,AesPleasingAsianGirls,Low-Day1071,Precious,,,/r/AesPleasingAsianGirls/comments/13huk92/prec...,https://www.reddit.com/gallery/13huk92,13huk92.jpg,,,False,False,False,[]
13i312w,13i312w,AesPleasingAsianGirls,Defiant-Appearance64,Yoo Sul Young,,,/r/AesPleasingAsianGirls/comments/13i312w/yoo_...,https://v3.redgifs.com/watch/excellentoutrageo...,13i312w.jpg,,,False,False,False,[]
13i8lv5,13i8lv5,AesPleasingAsianGirls,sandalsanglir2,Balinese Singer,,,/r/AesPleasingAsianGirls/comments/13i8lv5/bali...,https://www.reddit.com/gallery/13i8lv5,13i8lv5.jpg,,,False,False,False,[]


CPU times: total: 3.88 s
Wall time: 6.06 s


In [11]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]
filtered.dropna(axis=1, how='all')
filtered.reset_index(drop=True, inplace=True)
filtered.set_index('id', inplace=True, drop=False)
display("==== FILTERED DATA ====")
display(filtered.shape)
display(filtered)

'==== FILTERED DATA ===='

(1244, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13azmti,13azmti,SFWRedheads,Scarlett_somebody,I’m going out for the first time in so long! W...,,,/r/SFWRedheads/comments/13azmti/im_going_out_f...,https://i.redd.it/ktflvdzo1iya1.jpg,13azmti.jpg,,,False,False,False,[]
13b0cv0,13b0cv0,SFWRedheads,RedWicked91,Up for a trip?😁,,,/r/SFWRedheads/comments/13b0cv0/up_for_a_trip/,https://i.redgifs.com/i/neighboringmediumgolde...,13b0cv0.jpg,,,False,False,False,[]
13b2ghi,13b2ghi,SFWRedheads,Powerful_Frame9794,Good morning loves! Like my braids again?,,,/r/SFWRedheads/comments/13b2ghi/good_morning_l...,https://i.redd.it/cpe6hy2h2hya1.jpg,13b2ghi.jpg,,,False,False,False,[]
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,,False,False,False,[]
13dcf19,13dcf19,SFWRedheads,Scarlett_somebody,"Messy hair, cannot care 😹💚",,,/r/SFWRedheads/comments/13dcf19/messy_hair_can...,https://i.redd.it/alhlwwlp6yya1.jpg,13dcf19.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13g70ma,13g70ma,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13g70ma/song...,https://i.redd.it/m86fb2tugkza1.jpg,13g70ma.jpg,,,False,False,False,[]
13g9d43,13g9d43,AesPleasingAsianGirls,ExperienceOver7707,KitKat,,,/r/AesPleasingAsianGirls/comments/13g9d43/kitkat/,https://i.redd.it/bekk940j2lza1.jpg,13g9d43.jpg,,,False,False,False,[]
13h4frm,13h4frm,AesPleasingAsianGirls,No_Slide3921,Wwyd if you had a date with elfie?,,,/r/AesPleasingAsianGirls/comments/13h4frm/wwyd...,https://i.redd.it/c08d0u2n6sza1.jpg,13h4frm.jpg,,,False,False,False,[]


CPU times: total: 0 ns
Wall time: 120 ms


In [12]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 1244/1244 [00:00<00:00, 47749.34it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Model =='

(1244, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13azmti,13azmti,SFWRedheads,Scarlett_somebody,I’m going out for the first time in so long! W...,,,/r/SFWRedheads/comments/13azmti/im_going_out_f...,https://i.redd.it/ktflvdzo1iya1.jpg,13azmti.jpg,,RedHeadDiffusion,False,False,False,[]
13b0cv0,13b0cv0,SFWRedheads,RedWicked91,Up for a trip?😁,,,/r/SFWRedheads/comments/13b0cv0/up_for_a_trip/,https://i.redgifs.com/i/neighboringmediumgolde...,13b0cv0.jpg,,RedHeadDiffusion,False,False,False,[]
13b2ghi,13b2ghi,SFWRedheads,Powerful_Frame9794,Good morning loves! Like my braids again?,,,/r/SFWRedheads/comments/13b2ghi/good_morning_l...,https://i.redd.it/cpe6hy2h2hya1.jpg,13b2ghi.jpg,,RedHeadDiffusion,False,False,False,[]
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13dcf19,13dcf19,SFWRedheads,Scarlett_somebody,"Messy hair, cannot care 😹💚",,,/r/SFWRedheads/comments/13dcf19/messy_hair_can...,https://i.redd.it/alhlwwlp6yya1.jpg,13dcf19.jpg,,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13g70ma,13g70ma,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13g70ma/song...,https://i.redd.it/m86fb2tugkza1.jpg,13g70ma.jpg,,SexyAsianDiffusion,False,False,False,[]
13g9d43,13g9d43,AesPleasingAsianGirls,ExperienceOver7707,KitKat,,,/r/AesPleasingAsianGirls/comments/13g9d43/kitkat/,https://i.redd.it/bekk940j2lza1.jpg,13g9d43.jpg,,SexyAsianDiffusion,False,False,False,[]
13h4frm,13h4frm,AesPleasingAsianGirls,No_Slide3921,Wwyd if you had a date with elfie?,,,/r/AesPleasingAsianGirls/comments/13h4frm/wwyd...,https://i.redd.it/c08d0u2n6sza1.jpg,13h4frm.jpg,,SexyAsianDiffusion,False,False,False,[]


CPU times: total: 46.9 ms
Wall time: 69.8 ms


In [13]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 1244/1244 [11:52<00:00,  1.75it/s]
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Path =='

(1244, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13azmti,13azmti,SFWRedheads,Scarlett_somebody,I’m going out for the first time in so long! W...,,,/r/SFWRedheads/comments/13azmti/im_going_out_f...,https://i.redd.it/ktflvdzo1iya1.jpg,13azmti.jpg,data/image/13azmti.jpg,RedHeadDiffusion,False,False,False,[]
13b0cv0,13b0cv0,SFWRedheads,RedWicked91,Up for a trip?😁,,,/r/SFWRedheads/comments/13b0cv0/up_for_a_trip/,https://i.redgifs.com/i/neighboringmediumgolde...,13b0cv0.jpg,,RedHeadDiffusion,False,False,False,[]
13b2ghi,13b2ghi,SFWRedheads,Powerful_Frame9794,Good morning loves! Like my braids again?,,,/r/SFWRedheads/comments/13b2ghi/good_morning_l...,https://i.redd.it/cpe6hy2h2hya1.jpg,13b2ghi.jpg,data/image/13b2ghi.jpg,RedHeadDiffusion,False,False,False,[]
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13dcf19,13dcf19,SFWRedheads,Scarlett_somebody,"Messy hair, cannot care 😹💚",,,/r/SFWRedheads/comments/13dcf19/messy_hair_can...,https://i.redd.it/alhlwwlp6yya1.jpg,13dcf19.jpg,data/image/13dcf19.jpg,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13g70ma,13g70ma,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13g70ma/song...,https://i.redd.it/m86fb2tugkza1.jpg,13g70ma.jpg,data/image/13g70ma.jpg,SexyAsianDiffusion,False,False,False,[]
13g9d43,13g9d43,AesPleasingAsianGirls,ExperienceOver7707,KitKat,,,/r/AesPleasingAsianGirls/comments/13g9d43/kitkat/,https://i.redd.it/bekk940j2lza1.jpg,13g9d43.jpg,data/image/13g9d43.jpg,SexyAsianDiffusion,False,False,False,[]
13h4frm,13h4frm,AesPleasingAsianGirls,No_Slide3921,Wwyd if you had a date with elfie?,,,/r/AesPleasingAsianGirls/comments/13h4frm/wwyd...,https://i.redd.it/c08d0u2n6sza1.jpg,13h4frm.jpg,data/image/13h4frm.jpg,SexyAsianDiffusion,False,False,False,[]


CPU times: total: 51.4 s
Wall time: 13min 5s


In [14]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 1244/1244 [00:00<00:00, 3461.57it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Exists =='

(1244, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13azmti,13azmti,SFWRedheads,Scarlett_somebody,I’m going out for the first time in so long! W...,,,/r/SFWRedheads/comments/13azmti/im_going_out_f...,https://i.redd.it/ktflvdzo1iya1.jpg,13azmti.jpg,data/image/13azmti.jpg,RedHeadDiffusion,True,False,False,[]
13b0cv0,13b0cv0,SFWRedheads,RedWicked91,Up for a trip?😁,,,/r/SFWRedheads/comments/13b0cv0/up_for_a_trip/,https://i.redgifs.com/i/neighboringmediumgolde...,13b0cv0.jpg,,RedHeadDiffusion,False,False,False,[]
13b2ghi,13b2ghi,SFWRedheads,Powerful_Frame9794,Good morning loves! Like my braids again?,,,/r/SFWRedheads/comments/13b2ghi/good_morning_l...,https://i.redd.it/cpe6hy2h2hya1.jpg,13b2ghi.jpg,data/image/13b2ghi.jpg,RedHeadDiffusion,True,False,False,[]
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13dcf19,13dcf19,SFWRedheads,Scarlett_somebody,"Messy hair, cannot care 😹💚",,,/r/SFWRedheads/comments/13dcf19/messy_hair_can...,https://i.redd.it/alhlwwlp6yya1.jpg,13dcf19.jpg,data/image/13dcf19.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13g70ma,13g70ma,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13g70ma/song...,https://i.redd.it/m86fb2tugkza1.jpg,13g70ma.jpg,data/image/13g70ma.jpg,SexyAsianDiffusion,True,False,False,[]
13g9d43,13g9d43,AesPleasingAsianGirls,ExperienceOver7707,KitKat,,,/r/AesPleasingAsianGirls/comments/13g9d43/kitkat/,https://i.redd.it/bekk940j2lza1.jpg,13g9d43.jpg,data/image/13g9d43.jpg,SexyAsianDiffusion,True,False,False,[]
13h4frm,13h4frm,AesPleasingAsianGirls,No_Slide3921,Wwyd if you had a date with elfie?,,,/r/AesPleasingAsianGirls/comments/13h4frm/wwyd...,https://i.redd.it/c08d0u2n6sza1.jpg,13h4frm.jpg,data/image/13h4frm.jpg,SexyAsianDiffusion,True,False,False,[]


CPU times: total: 156 ms
Wall time: 453 ms


In [15]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 1244/1244 [00:07<00:00, 169.09it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Hash =='

(1244, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13azmti,13azmti,SFWRedheads,Scarlett_somebody,I’m going out for the first time in so long! W...,,1d176a4d01d33d41d1e333a883ad37b5,/r/SFWRedheads/comments/13azmti/im_going_out_f...,https://i.redd.it/ktflvdzo1iya1.jpg,13azmti.jpg,data/image/13azmti.jpg,RedHeadDiffusion,True,False,False,[]
13b0cv0,13b0cv0,SFWRedheads,RedWicked91,Up for a trip?😁,,,/r/SFWRedheads/comments/13b0cv0/up_for_a_trip/,https://i.redgifs.com/i/neighboringmediumgolde...,13b0cv0.jpg,,RedHeadDiffusion,False,False,False,[]
13b2ghi,13b2ghi,SFWRedheads,Powerful_Frame9794,Good morning loves! Like my braids again?,,d0d31e82e07a8ef0ca177c1b8657a5ec,/r/SFWRedheads/comments/13b2ghi/good_morning_l...,https://i.redd.it/cpe6hy2h2hya1.jpg,13b2ghi.jpg,data/image/13b2ghi.jpg,RedHeadDiffusion,True,False,False,[]
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13dcf19,13dcf19,SFWRedheads,Scarlett_somebody,"Messy hair, cannot care 😹💚",,3b6546176b2d9e39c53fd0674ed7c734,/r/SFWRedheads/comments/13dcf19/messy_hair_can...,https://i.redd.it/alhlwwlp6yya1.jpg,13dcf19.jpg,data/image/13dcf19.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13g70ma,13g70ma,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,c0ae1c6550ea4a077ed704c04f6ecaf7,/r/AesPleasingAsianGirls/comments/13g70ma/song...,https://i.redd.it/m86fb2tugkza1.jpg,13g70ma.jpg,data/image/13g70ma.jpg,SexyAsianDiffusion,True,False,False,[]
13g9d43,13g9d43,AesPleasingAsianGirls,ExperienceOver7707,KitKat,,2594e389e9b167cd7cc996c5be6302a9,/r/AesPleasingAsianGirls/comments/13g9d43/kitkat/,https://i.redd.it/bekk940j2lza1.jpg,13g9d43.jpg,data/image/13g9d43.jpg,SexyAsianDiffusion,True,False,False,[]
13h4frm,13h4frm,AesPleasingAsianGirls,No_Slide3921,Wwyd if you had a date with elfie?,,d5d369461053f80a685cadf0fbbc9e37,/r/AesPleasingAsianGirls/comments/13h4frm/wwyd...,https://i.redd.it/c08d0u2n6sza1.jpg,13h4frm.jpg,data/image/13h4frm.jpg,SexyAsianDiffusion,True,False,False,[]


CPU times: total: 1.8 s
Wall time: 7.54 s


In [16]:
%%time
import torch
if torch.cuda.is_available():
	caption_0 = BlipCaption("cuda")
	caption_1 = BlipCaption("cuda")
else:
	caption_0 = BlipCaption("cpu")
	caption_1 = BlipCaption("cpu")

C:\Users\ajsta\PycharmProjects\simple-image-collection\venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ajsta\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CPU times: total: 41.9 s
Wall time: 19min 19s


In [17]:
%%time

filtered['caption'] = filtered.progress_apply(lambda x: functions.apply_caption(x, [caption_0, caption_1]), axis=1)

display("== Filtered Data With Caption ==")
display(filtered.shape)
display(filtered)

Progress:   0%|          | 4/1244 [00:02<12:13,  1.69it/s]

"slow_conv2d_cpu" not implemented for 'Half'
"slow_conv2d_cpu" not implemented for 'Half'


Progress:   0%|          | 6/1244 [00:03<07:37,  2.71it/s]

"slow_conv2d_cpu" not implemented for 'Half'


Progress:   1%|          | 8/1244 [00:04<11:21,  1.81it/s]

...

Progress:  10%|▉         | 121/1244 [00:53<09:16,  2.02it/s]

"slow_conv2d_cpu" not implemented for 'Half'


Progress:  10%|▉         | 123/1244 [00:54<07:47,  2.40it/s]

"slow_conv2d_cpu" not implemented for 'Half'
"slow_conv2d_cpu" not implemented for 'Half'


Progress:  10%|▉         | 124/1244 [00:55<10:17,  1.81it/s]

In [18]:
%%time

filtered_more = filtered.loc[(filtered['caption'] != "") & (filtered['caption'].notnull()) & (filtered['exists'] == True)]

display("== Filtered On Missing Caption ==")
display(filtered_more.shape)
display(filtered_more)

In [19]:
%%time

dropped = filtered_more.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)

display("== Dropped And Reset Data ==")
display(dropped.shape)
display(dropped)

In [20]:
%%time

current = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system)

display(current.shape)
display(current)

In [21]:
%%time

temp_current = current
temp_current.set_index('id', inplace=True, drop=False)

temp_new = dropped
temp_new.set_index('id', inplace=True, drop=False)

temp_new.loc[~temp_new.index.isin(temp_current.index)]

In [22]:
%%time

concat = pd.concat([current, dropped])

display("== Concatenated Data ==")
display(concat.shape)
display(concat)

In [23]:
%%time

file_list_ = file_system.ls("data/image")
concat['path'] = concat.progress_apply(lambda x: functions.fix_path(x, file_list), axis=1)

display("== Concatenated Data With Path ==")
display(concat.shape)
display(concat)

In [24]:
%%time

back_up_name = f"data/parquet/back_{dt.datetime.timestamp(dt.datetime.now())}.parquet"
display(f"== Writing Back-Up {back_up_name} ==")
current.to_parquet(back_up_name, schema=schema, filesystem=file_system)

In [25]:
%%time

concat.to_parquet("data/parquet/back.parquet", schema=schema, filesystem=file_system)
new = pd.read_parquet("data/parquet/back.parquet", engine='pyarrow', schema=schema, filesystem=file_system)

display("== New Data ==")
display(new.shape)
display(new)

In [26]:
!jupyter notebook stop